In [1]:
from pathlib import Path
from SDART import SDART, SDARTConfig

SOURCE_SHAPEFILES = Path("./SDART_TEST/data/Source_Shapefiles")
UPDATED_SHAPEFILES = Path("./SDART_TEST/SDART_TEST.gdb")
SCRATCH_GDB = Path("./SDART_TEST/data/SDART_Scratch.gdb")

cfg = SDARTConfig(
    source_shapefiles=SOURCE_SHAPEFILES,
    updated_shapefiles=UPDATED_SHAPEFILES,
    scratch_gdb=SCRATCH_GDB,
)

s = SDART(cfg)


In [2]:
# After creating `s = SDART(cfg)`
inv = s.inventory_table()
inv  # display

,Layer Key,Updated Expected FC,Updated Status,Source Expected Shapefile,Source Status,Resolved Updated Path,Resolved Source Path
0,junctions,Updated_junctions,Found,Source_junctions.shp,Found,SDART_TEST/SDART_TEST.gdb/Updated_junctions,SDART_TEST/data/Source_Shapefiles/Source_junct...
1,storages,Updated_storages,Missing,Source_storages.shp,Found,,SDART_TEST/data/Source_Shapefiles/Source_stora...
2,dividers,Updated_dividers,Found,Source_dividers.shp,Found,SDART_TEST/SDART_TEST.gdb/Updated_dividers,SDART_TEST/data/Source_Shapefiles/Source_divid...
3,outfalls,Updated_outfalls,Found,Source_outfalls.shp,Found,SDART_TEST/SDART_TEST.gdb/Updated_outfalls,SDART_TEST/data/Source_Shapefiles/Source_outfa...
4,conduits,Updated_conduits,Found,Source_conduits.shp,Found,SDART_TEST/SDART_TEST.gdb/Updated_conduits,SDART_TEST/data/Source_Shapefiles/Source_condu...
5,orifices,Updated_orifices,Found,Source_orifices.shp,Found,SDART_TEST/SDART_TEST.gdb/Updated_orifices,SDART_TEST/data/Source_Shapefiles/Source_orifi...
6,outlets,Updated_outlets,Found,Source_outlets.shp,Found,SDART_TEST/SDART_TEST.gdb/Updated_outlets,SDART_TEST/data/Source_Shapefiles/Source_outle...
7,pumps,Updated_pumps,Found,Source_pumps.shp,Found,SDART_TEST/SDART_TEST.gdb/Updated_pumps,SDART_TEST/data/Source_Shapefiles/Source_pumps...
8,weirs,Updated_weirs,Found,Source_weirs.shp,Found,SDART_TEST/SDART_TEST.gdb/Updated_weirs,SDART_TEST/data/Source_Shapefiles/Source_weirs...
9,subcatchments,Updated_subcatchments,Missing,Source_subcatchments.shp,Found,,SDART_TEST/data/Source_Shapefiles/Source_subca...


In [4]:
# 02_backups.ipynb — safe copies (no .lock files copied)
s.backup(out_root=Path("./SDART_TEST/_backups"), overwrite=False)


INFO: Backups created → SDART_TEST\_backups\GIS_Backup_20250907_201013 , SDART_TEST\_backups\Source_Backup


(WindowsPath('SDART_TEST/_backups/GIS_Backup_20250907_201013'),
 WindowsPath('SDART_TEST/_backups/Source_Backup'))

In [5]:
# 03_topology.ipynb — merge & split; snap optional + explicit radius
# snap=True/False, radius in feet (ArcGIS converts if your data are meters)
s.prepare_topology(snap=True, snap_radius_ft=5.0)


INFO: [Updated_conduits] Unsplit → SDART_TEST\data\SDART_Scratch.gdb/<Updated_conduits_unsplit>
INFO: [Updated_conduits] Snap points → unsplit @ 5.0 ft
INFO: [Updated_conduits] Snap points → unsplit @ 5.0 ft
INFO: [Updated_conduits] Snap points → unsplit @ 5.0 ft
INFO: [Updated_conduits] Split at Updated_junctions → step 1
INFO: [Updated_conduits] Split at Updated_dividers → step 2
INFO: [Updated_conduits] Split at Updated_outfalls → step 3
INFO: [Updated_conduits] Done → SDART_TEST/SDART_TEST.gdb/Updated_conduits
INFO: [Updated_orifices] Unsplit → SDART_TEST\data\SDART_Scratch.gdb/<Updated_orifices_unsplit>
INFO: [Updated_orifices] Snap points → unsplit @ 5.0 ft
INFO: [Updated_orifices] Snap points → unsplit @ 5.0 ft
INFO: [Updated_orifices] Snap points → unsplit @ 5.0 ft
INFO: [Updated_orifices] Split at Updated_junctions → step 1
INFO: [Updated_orifices] Split at Updated_dividers → step 2
INFO: [Updated_orifices] Split at Updated_outfalls → step 3
INFO: [Updated_orifices] Done → SDA

In [6]:
# 04_shift.ipynb — compute dx,dy from one Source point to one Updated point; shift ALL Source_*.shp
dx, dy = s.shift_sources_by_two_oids(
    moving_feature_label="Feature 0",      # from Source_junctions.shp
    anchor_feature_label="Feature 1",      # from Updated_junctions
    moving_points="Source_junctions.shp",
    anchor_points_fc="Updated_junctions",
)
dx, dy


INFO: Computed shift dx=702.661, dy=-985.863
INFO: Shifting Source_conduits.shp (polyline) by dx=702.661, dy=-985.863
INFO: Shifting Source_dividers.shp (point) by dx=702.661, dy=-985.863
INFO: Shifting Source_junctions.shp (point) by dx=702.661, dy=-985.863
INFO: Shifting Source_orifices.shp (polyline) by dx=702.661, dy=-985.863
INFO: Shifting Source_outfalls.shp (point) by dx=702.661, dy=-985.863
INFO: Shifting Source_outlets.shp (polyline) by dx=702.661, dy=-985.863
INFO: Shifting Source_pumps.shp (polyline) by dx=702.661, dy=-985.863
INFO: Shifting Source_storages.shp (point) by dx=702.661, dy=-985.863
INFO: Shifting Source_subcatchments.shp (polygon) by dx=702.661, dy=-985.863
INFO: Shifting Source_weirs.shp (polyline) by dx=702.661, dy=-985.863
INFO: All Source_* layers shifted.


(702.66091170453, -985.8627056134865)

In [ ]:
# %%
# Universal inputs (tweak as needed)
POINTS_LAYERS = ("junctions", "storages", "dividers", "outfalls")
LINE_LAYERS   = ("conduits", "orifices", "outlets", "pumps", "weirs")

# Points: grid-search radii (ft)
POINTS_RADII  = (1, 2, 3, 5)

# Lines: used for CENTER/LARGEST; CLOSEST is unlimited inside SDART
LINE_RADIUS   = 1

# Geometry validation thresholds
TOLERANCES = {
    "len":    10.0,   # allowable length difference in feet
    "angle":  12.0,   # degrees; 180° reversal treated as 0° in SDART
    "curves": 10.0,   # curve-count tolerance
    "verts":  9999.0  # effectively ignore vertex-count differences
    # If your SDART supports it, you can also add: "len_ratio": 0.30
}

# Run the package (xlsx_path=None => SDART uses its default path)
xlsx_out = s.spatial_join_package_to_excel(
    points_layers=POINTS_LAYERS,
    line_layers=LINE_LAYERS,
    points_radii=POINTS_RADII,
    line_radius=LINE_RADIUS,
    tolerances=TOLERANCES,
    xlsx_path=None,
    apply_updates=True
)

print(f"Workbook written → {xlsx_out}")


INFO: [junctions] Points SpatialJoin applied in-place at 1 ft → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_junctions
INFO: Skip points layer storages because inputs are missing
INFO: [dividers] Points SpatialJoin applied in-place at 1 ft → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_dividers
INFO: [outfalls] Points SpatialJoin applied in-place at 1 ft → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_outfalls
INFO: [conduits] De-duplicated 4 features mapped to the same SWMM element; will clear them to NULL.
INFO: [conduits] Line attributes applied in-place from sweep winners → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_conduits (updated 5 features; cleared 0 duplicates)
INFO: [orifices] Line attributes applied in-place from sweep winners → A:/OneDrive - LEO A DALY/Pycharm

In [3]:
# Fill nulls for points + conduits with stable placeholders
s.fill_null_names_unknown(include_points=True, include_conduits=True)

INFO: [junctions] Filled 5 NULL Names → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_junctions
INFO: [dividers] Filled 0 NULL Names → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_dividers
INFO: [outfalls] Filled 0 NULL Names → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_outfalls
INFO: [conduits] Filled 4 NULL Names → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_conduits


In [4]:
# 2) Assign FromNode/ToNode using 2 ft search radius (tweak if needed).
s.assign_from_to_nodes(search_radius_ft=2.0)

INFO: [junctions] Filled 0 NULL Names → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_junctions
INFO: [dividers] Filled 0 NULL Names → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_dividers
INFO: [outfalls] Filled 0 NULL Names → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_outfalls
INFO: [conduits] From/To assigned within 2.0 Feet → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_conduits (updated 6 features)
INFO: [orifices] From/To assigned within 2.0 Feet → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_orifices (updated 1 features)
INFO: [outlets] From/To assigned within 2.0 Feet → A:/OneDrive - LEO A DALY/Pycharm Projects/GIS-to-SWMM-conversion/SDART_TEST/SDART_TEST.gdb/Updated_outlets (updated 0 features)
INFO: [pumps] Fr